In [38]:
import numpy as np
import pandas as pd

# Load and prepare data
df = pd.read_csv("sales forecasting.csv", encoding="ISO-8859-1")
df['Order Date'] = pd.to_datetime(df['Order Date'], errors='coerce')
daily_sales = df.groupby('Order Date')['Sales'].sum().sort_index()
daily_sales = daily_sales.asfreq('D', fill_value=0)

# Normalize
sales_values = daily_sales.values
min_val, max_val = sales_values.min(), sales_values.max()
normalized = (sales_values - min_val) / (max_val - min_val)

# Create sequences
def create_sequences(data, seq_len=7):
    X, y = [], []
    for i in range(len(data) - seq_len):
        X.append(data[i:i+seq_len])
        y.append(data[i+seq_len])
    return np.array(X), np.array(y)

X, y = create_sequences(normalized, 7)
X = X.reshape(X.shape[0], X.shape[1], 1)

In [40]:
import numpy as np

# Sigmoid function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Derivative of sigmoid function
def sigmoid_derivative(x):
    return x * (1 - x)

# LSTM cell
class LSTM:
    def __init__(self, input_size, hidden_size):
        self.input_size = input_size
        self.hidden_size = hidden_size

        # Initialize weights
        self.Wf = np.random.rand(input_size + hidden_size, hidden_size)
        self.Wi = np.random.rand(input_size + hidden_size, hidden_size)
        self.Wg = np.random.rand(input_size + hidden_size, hidden_size)
        self.Wo = np.random.rand(input_size + hidden_size, hidden_size)

    def forward(self, x, h_prev, c_prev):
        # Concatenate input and previous hidden state
        xh = np.concatenate((x, h_prev), axis=0)

        # Forget gate
        f = sigmoid(np.dot(xh, self.Wf))

        # Input gate
        i = sigmoid(np.dot(xh, self.Wi))

        # Cell gate
        g = np.tanh(np.dot(xh, self.Wg))

        # Output gate
        o = sigmoid(np.dot(xh, self.Wo))

        # Update cell state
        c = f * c_prev + i * g

        # Update hidden state
        h = o * np.tanh(c)

        return h, c

# Example usage
input_size = 1
hidden_size = 16

lstm = LSTM(input_size=input_size, hidden_size=hidden_size)

# Initialize hidden and cell states
h_prev = np.zeros((hidden_size,))
c_prev = np.zeros((hidden_size,))

# Generate random input
x = np.random.rand(input_size,)

# Forward pass
h, c = lstm.forward(x, h_prev, c_prev)
print("Hidden state:", h)
print("Cell state:", c)


Hidden state: [0.12931404 0.08989726 0.0701401  0.05400055 0.02155306 0.15492978
 0.05927417 0.10189136 0.01841469 0.01789792 0.08961239 0.08743005
 0.16708091 0.053004   0.14739358 0.02405193]
Cell state: [0.23221448 0.14157297 0.1226107  0.08909395 0.03694372 0.27221081
 0.09326609 0.20492694 0.03104776 0.03570911 0.17700488 0.14967303
 0.27878508 0.08282326 0.30164862 0.04063881]


In [42]:
import numpy as np
import pandas as pd

# Load Data
def load_data(file_path, sequence_length):
    df = pd.read_csv(file_path)
    sales = df['Sales'].values.astype(float)  # Assuming 'Sales' column contains values
    
    # Normalize data
    min_val, max_val = min(sales), max(sales)
    sales = (sales - min_val) / (max_val - min_val)
    
    X, y = [], []
    for i in range(len(sales) - sequence_length):
        X.append(sales[i:i + sequence_length])
        y.append(sales[i + sequence_length])
    
    return np.array(X), np.array(y), min_val, max_val

# Sigmoid Function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# LSTM Class (Basic)
class LSTM:
    def __init__(self, input_size, hidden_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        # Weight matrices
        self.Wf = np.random.randn(hidden_size, input_size + hidden_size)
        self.bf = np.zeros(hidden_size)
        
        self.Wi = np.random.randn(hidden_size, input_size + hidden_size)
        self.bi = np.zeros(hidden_size)
        
        self.Wc = np.random.randn(hidden_size, input_size + hidden_size)
        self.bc = np.zeros(hidden_size)
        
        self.Wo = np.random.randn(hidden_size, input_size + hidden_size)
        self.bo = np.zeros(hidden_size)
    
    def forward(self, x, h_prev, c_prev):
        """ Forward pass for one time step """
        combined = np.concatenate((h_prev, x))
        
        forget_gate = sigmoid(np.dot(self.Wf, combined) + self.bf)
        input_gate = sigmoid(np.dot(self.Wi, combined) + self.bi)
        candidate_cell = np.tanh(np.dot(self.Wc, combined) + self.bc)
        output_gate = sigmoid(np.dot(self.Wo, combined) + self.bo)
        
        c_next = forget_gate * c_prev + input_gate * candidate_cell
        h_next = output_gate * np.tanh(c_next)
        
        return h_next, c_next

# Training Function
def train_lstm(X, y, hidden_size, epochs=100, learning_rate=0.01):
    lstm = LSTM(X.shape[1], hidden_size)
    h_prev, c_prev = np.zeros(hidden_size), np.zeros(hidden_size)
    
    for epoch in range(epochs):
        total_loss = 0
        
        for i in range(len(X)):
            h_prev, c_prev = lstm.forward(X[i], h_prev, c_prev)
            loss = (h_prev - y[i]) ** 2
            total_loss += np.sum(loss)
        
        # Update weights (Gradient Descent Approximation)
        lstm.Wf -= learning_rate * np.random.randn(*lstm.Wf.shape)
        lstm.Wi -= learning_rate * np.random.randn(*lstm.Wi.shape)
        lstm.Wc -= learning_rate * np.random.randn(*lstm.Wc.shape)
        lstm.Wo -= learning_rate * np.random.randn(*lstm.Wo.shape)
        
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}")
    
    return lstm

# Forecast Function
def forecast_sales(lstm, X_test):
    h_prev, c_prev = np.zeros(lstm.hidden_size), np.zeros(lstm.hidden_size)
    predictions = []
    
    for i in range(len(X_test)):
        h_prev, c_prev = lstm.forward(X_test[i], h_prev, c_prev)
        predictions.append(h_prev)
    
    return np.array(predictions)

# Main Execution
file_path = "sales forecasting.csv"  # Update path if needed
sequence_length = 5
hidden_size = 10

X, y, min_val, max_val = load_data(file_path, sequence_length)

# Split Data
train_size = int(0.8 * len(X))
X_train, y_train = X[:train_size], y[:train_size]
X_test, y_test = X[train_size:], y[train_size:]

# Train Model
lstm_model = train_lstm(X_train, y_train, hidden_size)

# Predict Future Sales
predicted_sales = forecast_sales(lstm_model, X_test)
predicted_sales = predicted_sales * (max_val - min_val) + min_val  # Reverse Normalization
y_test = y_test * (max_val - min_val) + min_val

# Show Results
print("Predicted Sales:", predicted_sales)
print("Actual Sales:", y_test)

Epoch 1/100, Loss: 1392.9609
Epoch 2/100, Loss: 1441.4066
Epoch 3/100, Loss: 1518.3704
Epoch 4/100, Loss: 1391.2007
Epoch 5/100, Loss: 1570.8774
Epoch 6/100, Loss: 1617.4063
Epoch 7/100, Loss: 1548.8744
Epoch 8/100, Loss: 1621.8069
Epoch 9/100, Loss: 1700.2074
Epoch 10/100, Loss: 1659.6344
Epoch 11/100, Loss: 1700.3568
Epoch 12/100, Loss: 1655.6430
Epoch 13/100, Loss: 1663.2223
Epoch 14/100, Loss: 1669.7197
Epoch 15/100, Loss: 1698.3547
Epoch 16/100, Loss: 1749.9348
Epoch 17/100, Loss: 1707.3213
Epoch 18/100, Loss: 1760.7199
Epoch 19/100, Loss: 1880.6658
Epoch 20/100, Loss: 1835.6423
Epoch 21/100, Loss: 1979.0822
Epoch 22/100, Loss: 1997.6672
Epoch 23/100, Loss: 2072.6606
Epoch 24/100, Loss: 2029.8936
Epoch 25/100, Loss: 2043.0836
Epoch 26/100, Loss: 1977.0367
Epoch 27/100, Loss: 1889.5033
Epoch 28/100, Loss: 1883.1983
Epoch 29/100, Loss: 1918.2290
Epoch 30/100, Loss: 1934.4998
Epoch 31/100, Loss: 2031.3751
Epoch 32/100, Loss: 1998.4761
Epoch 33/100, Loss: 2014.8715
Epoch 34/100, Loss: